In [1]:
import pandas as pd
import sys,time
import math

In [12]:
# Read '200_repos.csv' into DataFrame df
#
# NaN is assigned to empty cells
df = pd.read_csv('200_repos.csv')

In [ ]:
# subset dataframes for testing
# use .copy() as slicing will not allow for assignment
df10 = df.iloc[:10].copy()
df33 = df.iloc[:33].copy()

In [13]:
dfs = df[['CMC_id', 'source_code', 'forge']].copy()
dfc = pd.read_csv('project_criticality_all.csv')
num = 0
for row in dfs.itertuples():
    # only search if github
    if row.forge == 'github':
        # only search for strings; floats (NaN) are skipped
        if isinstance(row.source_code, str):
            url = str(row.source_code)
            # loop through df2 (criticality) looking for source code url
            for row2 in dfc.itertuples():
                if url == row2.url:
                    dfs.at[row.Index, 'criticality'] = row2.criticality_score
                    num += 1
                    break
            sys.stdout.write(".")
            sys.stdout.flush()
print(str(num), 'criticality scores found and updated')

.......................................................................................................................................................................62 criticality scores found and updated


In [15]:
# update MERGED sheet with new data
# 'CMC_id' is the key, drop 'repo', and 'forge' before the merge
# to prevent duplicate columns
dfs.drop(columns = ['source_code', 'forge'], inplace = True)
dfm = pd.merge(df,dfs,on = ['CMC_id'], how = 'outer')

In [20]:
# write out new data
dfs.to_csv('200_crit.csv', encoding='utf-8', index = 0)
dfm.to_csv('200_merged.csv', encoding='utf-8', index = 0)